## Imports

In [1]:
from geopy.geocoders import GoogleV3
from bs4 import BeautifulSoup
import requests
import time
import pandas as pd
import numpy as np
import urllib3
import re
from textwrap import shorten
import os
import geopandas as gpd

In [3]:
df = pd.read_csv('compass_office_data.csv')

In [9]:
df['address_2'] = df['address_2'].replace('\n',' ',regex=True)

In [11]:
df['full_address'] = df['address'] + ' ' + df['address_2']

In [12]:
%store -r google_maps_API_Key
geolocator = GoogleV3(api_key=google_maps_API_Key)

In [13]:
df['geo_address'] = df['full_address']
df['loc'] = df['geo_address'].apply(geolocator.geocode, timeout=10)
df['point'] = df['loc'].apply(lambda loc: tuple(loc.point) if loc else None)
df[['lat','lon','altitude']] = pd.DataFrame(df['point'].to_list(),index=df.index)

## Correction Section

## HTML popup formatter

In [6]:
df.columns

Index(['Unnamed: 0', 'MLS#', 'Unique', 'Final Agent/Team', 'agent_type',
       'TRD Notes', 'PROPERTY TYPE', 'ADDRESS', 'CITY', 'STATE OR PROVINCE',
       'ZIP OR POSTAL CODE', 'SOLD DATE', 'PRICE', 'URL', 'full_address',
       'geo_address', 'loc', 'point', 'lat', 'lon', 'altitude'],
      dtype='object')

In [31]:
def popup_html(row):
    i = row
    address = df['full_address'].iloc[i]
    office = df['name'].iloc[i]
    
    html = '''<!DOCTYPE html>
    <html>
    <head>
    <h4>{}</h4>'''.format(address) + '''
    </head>
    <body>
    <strong></strong><h4>{}<h4>'''.format(office) + '''<br> 
    </body>
    </html>
    '''
    return html


### HTML reservoir

In [8]:
#     Developer(s): <strong><em>{}'''.format(developer) + '''</strong></em><br><br>

## Map Maker

In [ ]:
import folium
from folium.plugins import MarkerCluster
import branca

f = folium.Figure(width=550, height=750)
m = folium.Map(location=df[["lat", "lon"]].mean().to_list(),zoom_start=3)

title_html = '''
              <h4 align="center" style="font-size:16px"><b>{}</b></h4>
             '''.format(f'Compass - August 2022')

marker_clusterer = MarkerCluster().add_to(m)

for i in range(0,len(df)):
    html = popup_html(i)
    iframe = branca.element.IFrame(html=html, width=510,height=280)
    popup = folium.Popup(folium.Html(html, script=True), max_width=510)
    folium.Marker([df['lat'].iloc[i],df['lon'].iloc[i]],
                 popup=popup).add_to(marker_clusterer)

m.get_root().html.add_child(folium.Element(title_html))
m.fit_bounds(bounds=df[['lat','lon']].mean().to_list(),max_zoom=15)
# m.add_to(f)
m

In [10]:
m.save('index.html')

In [11]:
pd.set_option('display.max_rows',1000)

In [16]:
df['ZIP OR POSTAL CODE'].value_counts().sum()

2234

In [12]:
x = df.groupby(['ZIP OR POSTAL CODE'])['Final Agent/Team'].value_counts().groupby(level=0).nlargest(3)
x.to_csv('busiest_brokers_by_zip_code_chicago.csv')
x

ZIP OR POSTAL CODE  ZIP OR POSTAL CODE  Final Agent/Team                   
60601.0             60601.0             Leila Zammatta                         20
                                        The Laricy Team                         4
                                        Grigory Pekarsky                        3
60602.0             60602.0             Grigory Pekarsky                        3
                                        Bari & Elena Team                       1
                                        Kate Waddell Group                      1
60603.0             60603.0             The Laricy Team                         1
                                        Weinberg Choi Residential               1
60605.0             60605.0             The Laricy Team                        13
                                        NL Ferrata Team                         9
                                        Grigory Pekarsky                        6
60606.0             60

## Map URL snagger

Map template URL: `https://trd-digital.github.io/trd-news-interactive-maps/{map-folder-name}`

In [13]:
base_name = 'https://trd-digital.github.io/trd-news-interactive-maps/'

In [14]:
cwd = os.getcwd()

cwd = cwd.split('/')

final_name = base_name + cwd[-1]
print(final_name)

https://trd-digital.github.io/trd-news-interactive-maps/Chicago_broker_rankings_08_2022
